# 

In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload all

In [2]:
import os
import sys
from typing import List
import numpy as np
import pandas as pd
import copy
from tqdm.notebook import tqdm

from weac.components import Segment, ScenarioConfig, WeakLayer, CriteriaConfig, Layer
from weac.analysis import CriteriaEvaluator

from layerwise.analysis.profile_utils import load_snowpilot_parsers, calc_avg_density_profile, eval_weac_over_layers, eval_weac_from_parser

In [ ]:
## Setup standard values

number_of_files = 10000
raw_data_dir = "../data/raw/snowpilot"
phi = 0.0
spacing = 100

scenario_config = ScenarioConfig(system_type="skier", phi=phi)
weaklayer = WeakLayer(rho=125, h=20, E=1.0, sigma_c=6.16, tau_c=5.09)
segments = [
    Segment(length=10000, has_foundation=True, m=0.0),
    Segment(
        length=10000,
        has_foundation=True,
        m=0.0,
    ),
]
criteria_config = CriteriaConfig()
criteria_evaluator = CriteriaEvaluator(criteria_config)

In [ ]:
## Parse all snowpilot files

paths, parsers = load_snowpilot_parsers(raw_data_dir, number_of_files, with_avalanche=False)

print(f"\nFound {len(paths)} files without avalanche")


Found 9665 files without avalanche


In [5]:
## Calculate average density profile

depth_profile, avg_density_profile, error_count = calc_avg_density_profile(parsers, spacing=spacing)
print("Error count:", error_count)
print("Average density profile:", avg_density_profile)

Error count: 177
Average density profile: [140.77341801 158.43864537 185.98716174 201.00289518 215.82784673
 223.00790056 232.33646296 234.81511694 240.2389327  242.55126309
 244.86440006 246.95444021 249.32526627 249.74003682 252.45738733
 253.22844853 256.25076133 260.47887999 270.96349909 274.08150636
 271.14563834 275.82679655 286.43290968 288.20882464 285.21545368
 293.65690087 293.96890666 299.22941338 305.80983265 302.87826073
 304.71286038 310.25000354 304.7175952  294.20544797 295.86090076
 310.38762966 312.47828707 312.47828707 320.47129935 328.99444903]


In [6]:
## Evaluate WEAC over layers

layers = []
for i, depth in enumerate(depth_profile):
    layers.append(Layer(rho=avg_density_profile[i], h=spacing))

data_rows = eval_weac_over_layers(layers, scenario_config, segments, weaklayer, criteria_evaluator, spacing=spacing)
dataframe = pd.DataFrame(data_rows)
dataframe.to_csv("../data/misc/average_density_profile.csv", index=False)

Processing weak layers:   0%|          | 0/40 [00:00<?, ?it/s]

In [7]:
import pandas as pd
import plotly.graph_objects as go

dataframe = pd.read_csv("../data/misc/average_density_profile.csv")
depths = dataframe["wl_depth"]
impact_criterion = dataframe["impact_criterion"]
coupled_criterion = dataframe["coupled_criterion"]
sserr_result = dataframe["sserr_result"]
touchdown_distance = dataframe["touchdown_distance"]

densities = [layer.rho for layer in layers]

fig = go.Figure()
fig.add_trace(go.Scatter(x=depths, y=impact_criterion, mode='lines', name='Impact Criterion'))
fig.add_trace(go.Scatter(x=depths, y=coupled_criterion, mode='lines', name='Coupled Criterion'))
fig.add_trace(go.Scatter(x=depths, y=sserr_result, mode='lines', name='SSERR'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=densities, y=depths, mode='lines', name='Density'))
fig.show()